# Introduction
![](https://translucentcomputing.com/wp-content/uploads/2019/12/performance.png)
While working in python jupyter notebooks using pandas with small datasets (100MB), performance degradation is not seen however when we work on large datasets (1 GB or above), performance issue of the notebook is clearly seen as it takes longer time for the cell to execute resulting in either longer execution time of the cell or the notebook failing due to insufficient memory.

Tools like Spark can handle large datasets (100's of GB to TB), taking full advantage of their capabilities usually requires more expensive hardware which is practically not possible while we try to work POC's or doing research. And unlike pandas, they lack rich feature sets for high quality data cleaning, exploration, and analysis. For medium-sized data, we’re better off trying to get more out of pandas, rather than switching to a different tool.

### Dataset

I have choosen a existing kaggle dataset from competition data namely **Jigsaw Multilingual Toxic Comment Classification** which has sizable data to demonstrate what I am trying to convey in this notebook.

#### Brief about the data:

Jigsaw's API, Perspective, serves toxicity models and others in a growing set of languages (see our documentation for the full list). Over the past year, the field has seen impressive multilingual capabilities from the latest model innovations, including few- and zero-shot learning. We're excited to learn whether these results "translate" (pun intended!) to toxicity classification. Your training data will be the English data provided for our previous two competitions and your test data will be Wikipedia talk page comments in several different languages.

As our computing resources and modeling capabilities grow, so does our potential to support healthy conversations across the globe. Develop strategies to build effective multilingual models and you'll help Conversation AI and the entire industry realize that potential.

#### Files

* jigsaw-toxic-comment-train.csv - data from our first competition. The dataset is made up of English comments from Wikipedia’s talk page edits.
* jigsaw-unintended-bias-train.csv - data from our second competition. This is an expanded version of the Civil Comments dataset with a range of additional labels.
* sample_submission.csv - a sample submission file in the correct format
* test.csv - comments from Wikipedia talk pages in different non-English languages.
* validation.csv - comments from Wikipedia talk pages in different non-English languages.
* jigsaw-toxic-comment-train-processed-seqlen128.csv - training data preprocessed for BERT
* jigsaw-unintended-bias-train-processed-seqlen128.csv - training data preprocessed for BERT
* validation-processed-seqlen128.csv - validation data preprocessed for BERT
* test-processed-seqlen128.csv - test data preprocessed for BERT

Let’s start by importing both pandas and our data in Python and taking a look at the first five rows.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
import gc
import subprocess

In [ ]:
jigsaw_seqlen128_df1 = pd.read_csv("../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv")

In [ ]:
jigsaw_seqlen128_df1.head()

In this notebook,I will demonstrating Python’s efficient RAM memory usage **techniques** with pandas, how to reduce a dataframe’s memory footprint by almost 90%!!!!

## Technique 1: Free Memory using gc.collect() 

In python notebook once a dataset loads into RAM it does not free on its own.So if you load a huge dataframe like above into pandas, and then make a copy of it and never use it again, that original dataframe will still be in RAM consuming memory which is same as other variables.Therefore we should adopt the habit of deleting dataframe when it is no longer in use to save memory.  

For example, suppose we create a dataframe by name `jigsaw_seqlen128_df1`, extract/explore some features and got the insights of the data but later on never used it ,then  `jigsaw_seqlen128_df1` will be consuming space.So we need to explicitely delete it using 'del' command.Note that there is no variable dependency on this dataframe before deleting it. So even after doing so there may be some residual memory usage.

That's where the garbage collection module comes into effect. `import gc` at the beginning of your project, and then each time you want to clear up space put command `gc.collect()` .  

It also helps to run `gc.collect()` after multiple transformations/functions/copying etc...  as all the little references/values accumulate.

Now we loaded the dataframe it is evident that it is consuming memory.Now let us delete the dataframe and collect residual garbage using gc.collect() method

In [ ]:
del jigsaw_seqlen128_df1
gc.collect()

## Technique 2: Datatype Conversions

We can use the info() method to fetch high level information about our dataframe, including its size, information about data types and memory usage.

By default, pandas approximates of the memory usage of the dataframe to save time. Because we’re interested in accuracy, we’ll set the memory_usage parameter to 'deep' to get an accurate number.

In [ ]:
jigsaw_seqlen128_df = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv')

In [ ]:
jigsaw_seqlen128_df.info(memory_usage='deep')

We can see that we have 1902194  rows and 47 columns. Pandas has automatically detected types for us, with 41 numeric columns and 6 object columns. Object columns are used for strings or where a column contains mixed data types.

So we can get a better understanding of where we can reduce this memory usage, let’s take a look into how Python and pandas store data in memory.


Each type has a specialized class in the pandas.core.internals module. Pandas uses the ObjectBlock class to represent the block containing string columns, and the FloatBlock class to represent the block containing float columns. For blocks representing numeric values like integers and floats, pandas combines the columns and stores them as a NumPy ndarray. The NumPy ndarray is built around a C array, and the values are stored in a contiguous block of memory. Due to this storage scheme, accessing a slice of values is incredibly fast.

Because each data type is stored separately, we are going to examine the memory usage by data type. Let’s start by looking at the average memory usage for data type.

In [ ]:
for dtype in ['float','int','object']:
    selected_dtype = jigsaw_seqlen128_df.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Immediately we can see that most of our memory is used by our 6 object columns. We’ll look at those later, but first lets see if we can improve on the memory usage for our numeric columns.

### Deeper understanding of subtypes

Under the hood pandas represents numeric values as NumPy ndarrays and stores them in a continuous block of memory. This storage model consumes less space and allows us to access the values themselves quickly. Because pandas represents each value of the same type using the same number of bytes, and a NumPy ndarray stores the number of values, pandas can return the number of bytes a numeric column consumes quickly and accurately.

Many types in pandas have multiple subtypes that can use fewer bytes to represent each value. For example, the float type has the float16, float32, and float64 subtypes. The number portion of a type’s name indicates the number of bits that type uses to represent values. For example, the subtypes we just listed use 2, 4, 8 and 16 bytes, respectively.

An int8 value uses 1 byte (or 8 bits) to store a value, and can represent 256 values (2^8) in binary. This means that we can use this subtype to represent values ranging from -128 to 127 (including 0).

We can use the numpy.iinfo class to verify the minimum and maximum values for each integer subtype. Let’s look at an example:

In [ ]:
int_types = ["uint8", "int8", "int16"]
for it in int_types:
    print(np.iinfo(it))

We can see here the difference between uint (unsigned integers) and int (signed integers). Both types have the same capacity for storage, but by only storing positive values, unsigned integers allow us to be more efficient with our storage of columns that only contain positive values.

### Numeric Column Optimization using Subtypes
We can use the function pd.to_numeric() to downcast our numeric types. We’ll use DataFrame.select_dtypes to select only the integer columns, then we’ll optimize the types and compare the memory usage.

In [ ]:
# We will be calculating memory usage a lot,so will create a function save some resource.
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)
gl_int = jigsaw_seqlen128_df.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(gl_int))
print(mem_usage(converted_int))
compare_ints = pd.concat([gl_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['Before','After']
compare_ints.apply(pd.Series.value_counts)

We can see a drop from 145.13 to 34.47 megabytes in memory usage, which is a more than 75% reduction. The overall impact on our original dataframe isn’t massive though, because there are so few integer columns.

Lets do the same thing with  float columns.

In [ ]:
gl_float = jigsaw_seqlen128_df.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric,downcast='float')
print(mem_usage(gl_float))
print(mem_usage(converted_float))
compare_floats = pd.concat([gl_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['Before','After']
compare_floats.apply(pd.Series.value_counts)

We can see that all our float columns were converted from float64 to float32, giving us a 50% reduction in memory usage.

Let’s create a copy of our original dataframe, assign these optimized numeric columns in place of the originals, and see what our overall memory usage is now.

In [ ]:
optimized_jigsaw_seqlen128_df = jigsaw_seqlen128_df.copy()
optimized_jigsaw_seqlen128_df[converted_int.columns] = converted_int
optimized_jigsaw_seqlen128_df[converted_float.columns] = converted_float
print(mem_usage(jigsaw_seqlen128_df))
print(mem_usage(optimized_jigsaw_seqlen128_df))

While we have dramatically reduced the memory usage of the numeric columns, overall see a 6-7 % reduction in the memory usage of our dataframe. Most of the gain will be observed while optimizing the object types.

Before we do, let’s take a closer look at how strings are stored in pandas compared to the numeric types
### Comparing String to Numeric storage
The object type represents values using Python string objects, partly due to the lack of support for missing string values in NumPy. Because Python is a high-level, interpreted language, it doesn’t have fine grained-control over how values in memory are stored.

This limitation causes strings to be stored in a fragmented way that consumes more memory and is slower to access. Each element in an object column is really a pointer that contains the “address” for the actual value’s location in memory.

Below is a diagram showing how numeric data is stored in NumPy data types vs how strings are stored using Python’s inbuilt types.

![](https://www.dataquest.io/wp-content/uploads/2019/01/numpy_vs_python.png)
As you might have noticed by now that object types consume considerable amount of memory. While each pointer takes up 1 byte of memory, each actual string value uses the same amount of memory that string would use if stored individually in Python. Let’s use sys.getsizeof() to prove that out, first by looking at individual strings, and then items in a pandas series.

In [ ]:
from sys import getsizeof
s1 = 'In progress'
s2 = 'memory consumption'
s3 = 'Objects in python is fun!'
s4 = 'Numerics consume less memory!'
for s in [s1, s2, s3, s4]:
    print(getsizeof(s))

In [ ]:
obj_ser = pd.Series(['In progress',
    'memory consumption',
    'Objects in python is fun!',
    'Numerics consume less memory!'])
obj_ser.apply(getsizeof)

From the above analogy it is observed that the size of strings when stored in a pandas series are identical to their usage as separate strings in Python.

### Object types optimization using categoricals

Pandas introduced Categoricals in version 0.15. The category type uses integer values under the hood to represent the values in a column, rather than the raw values. Pandas uses a separate mapping dictionary that maps the integer values to the raw ones. This arrangement is useful whenever a column contains a limited set of values. When we convert a column to the category dtype, pandas uses the most space efficient int subtype that can represent all of the unique values in a column.

To get an overview of where we might be able to use this type to reduce memory, let’s take a look at the number of unique values of each of our object types.

In [ ]:
jigsaw_seqlen128_df_obj = jigsaw_seqlen128_df.select_dtypes(include=['object']).copy()
jigsaw_seqlen128_df_obj.describe()

A quick glance reveals many columns where there are few unique values relative to the overall ~1902194 records in our data set.

Before we dive too far in, we’ll start by selecting just one of our object columns, and looking at what happens behind the scenes when we convert it to the categorical type. We will use the second column of our data set, rating.

Looking at the table above. we can see that it only contains two unique values. We’ll convert it to categorical by using the .astype() method.

In [ ]:
rating = jigsaw_seqlen128_df_obj.rating
print(rating.head())
rating_cat = rating.astype('category')
print(rating_cat.head())

As you can see, apart from the fact that the type of the column has changed, the data looks exactly the same. Let’s take a look under the hood at what’s happening.

In the following code, we use the Series.cat.codes attribute to return the integer values the category type uses to represent each value.

In [ ]:
rating_cat.head().cat.codes

You can see that each unique value has been assigned an integer, and that the underlying datatype for the column is now int8. This column doesn’t have any missing values, but if it did, the category subtype handles missing values by setting them to -1.

Lastly, let’s look at the memory usage for this column before and after converting to the category type.

In [ ]:
print(mem_usage(rating))
print(mem_usage(rating_cat))

We’ve gone from 117.91MB of memory usage to 1.81MB of memory usage, or a 99% reduction! Note that this particular column rating represents one of our best-case scenarios – a column with ~1902194 items of which there only 2 unique values.

While converting all of the columns to this type sounds appealing, it’s important to be aware of the trade-offs. The biggest one is the inability to perform numerical computations. We can’t do arithmetic with category columns or use methods like Series.min() and Series.max() without converting to a true numeric dtype first.

**We should stick to using the category type primarily for object columns where less than 50% of the values are unique.** If all of the values in a column are unique, the category type will end up using more memory. That’s because the column is storing all of the raw string values in addition to the integer category codes. You can read more about the limitations of the category type in the pandas documentation.

We’ll write a loop to iterate over each object column, check if the number of unique values is less than 50%, and if so, convert it to the category type.

In [ ]:
jigsaw_seqlen128_df_converted_obj = pd.DataFrame()
for col in jigsaw_seqlen128_df_obj.columns:
    num_unique_values = len(jigsaw_seqlen128_df_obj[col].unique())
    num_total_values = len(jigsaw_seqlen128_df_obj[col])
    if num_unique_values / num_total_values < 0.5:
        jigsaw_seqlen128_df_converted_obj.loc[:,col] = jigsaw_seqlen128_df_obj[col].astype('category')
    else:
        jigsaw_seqlen128_df_converted_obj.loc[:,col] = jigsaw_seqlen128_df_obj[col]
print(mem_usage(jigsaw_seqlen128_df_obj))
print(mem_usage(jigsaw_seqlen128_df_converted_obj))
compare_obj = pd.concat([jigsaw_seqlen128_df_obj.dtypes,jigsaw_seqlen128_df_converted_obj.dtypes],axis=1)
compare_obj.columns = ['Before','After']
compare_obj.apply(pd.Series.value_counts)

In this case, 50% of object columns were converted to the category type, however this won’t be the case with all data sets, so you should be sure to use the process above to check.But to demonstrate a better results I have used this technique in my latest kernel https://www.kaggle.com/pavansanagapati/ad-click-prediction-deep-interest-network-model.

What’s more, our memory usage for our object columns in this kernel example has gone from 3806.64MB to 2094.21MB, or a reduction of 45%. Let’s combine this with the rest of our dataframe and see where we sit in relation to the 4300MB memory usage we started with.

In [ ]:
optimized_jigsaw_seqlen128_df[jigsaw_seqlen128_df_converted_obj.columns] = jigsaw_seqlen128_df_converted_obj
mem_usage(optimized_jigsaw_seqlen128_df)

Wow, we have really made some progress by memory reduction of around 50% !!! 

So far, we have explored ways to reduce the memory footprint of an existing dataframe. By reading the dataframe in first and then iterating on ways to save memory, we were able to understand the amount of memory we can expect to save from each optimization better.

However in practice we often won’t have enough memory to represent all the values in a data set.

So the question is how do we apply memory-saving techniques when we can’t even create the dataframe in the first place? Let's explore and find out.

We can specify the optimal column types when we read the data set in. The pandas.read_csv() function has a few different parameters that allow us to do this. The dtype parameter accepts a dictionary that has (string) column names as the keys and NumPy type objects as the values.

First, we’ll store the final types of every column in a dictionary with keys for column names

In [ ]:
dtypes = optimized_jigsaw_seqlen128_df.dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
column_types = dict(zip(dtypes_col, dtypes_type))
# rather than print all 161 items, we'll
# sample 10 key/value pairs from the dict
# and print it nicely using prettyprint
preview = first2pairs = {key:value for key,value in list(column_types.items())[:10]}
print(preview)

Now we can use the dictionary, along with a few parameters for the date to read in the data with the correct types in a few lines:

In [ ]:
read_and_optimized = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv',dtype=column_types)
print(mem_usage(read_and_optimized))
read_and_optimized.head()

### There we go finally by optimizing the columns, we’ve managed to reduce the memory usage in pandas from 4300 MB to 2320.70 MB – an impressive 55% reduction!

# Technique 3: Import selective no of rows in data

Usually by default we read a csv file as a whole but there is an option called nrows where we can specify no of rows to import .For example if we want to read a csv file with the first 2000 rows only then syntax is as follows 

In [ ]:
train_df = pd.read_csv("../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv", nrows=20000)
train_df.head()

# Technique 4: Random Row Selection
This is nothing but performing a **random sampling** of the data and make best use of it.
For this we can build a list of random rows by writing a function 

In [ ]:
def csv_file_length(fname):
    process = subprocess.Popen(['wc', '-l', fname], stdout=subprocess.PIPE, 
                                              stderr=subprocess.PIPE)
    result, error = process.communicate()
    if process.returncode != 0:
        raise IOError(error)
    return int(result.strip().split()[0])

random_rows_selection = csv_file_length('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
print('Number of random rows in "jigsaw-toxic-comment-train.csv" is:', random_rows_selection)

Suppose we want to fetch a random sample of 10000 lines out of the total dataset. i.e we need to fetch a list of `lines - 1 - 10000` random numbers ranging from 1 to 796373. 

Note that while we are generating such a long list also takes a lot of space and  some time. So let us use the Technique 1 mentioned above and make sure to use del and gc.collect() when complete!!!!

Now let us generate list of lines to skip

In [ ]:
skip_rows = np.random.choice(np.arange(1, random_rows_selection), size=random_rows_selection-1-10000, replace=False)

Now let us sort the above list and print to check .....

In [ ]:
skip_rows=np.sort(skip_rows)
print('Rows to skip:', len(skip_rows))
print('Remaining rows in the random sample:', random_rows_selection-len(skip_rows))


Now that we got the random sample rows , let us fetch them from the csv file

In [ ]:
train = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv', skiprows=skip_rows)
train.head()

Now time for cleanup of unused list generated above and garbage collect.

In [ ]:
del skip_rows
gc.collect()

# Technique 5: Skip No of Rows
We can mention no of rows to skip by using the option of **"skiprows"** as shown below without heading.Note that plain skipping looses heading info. 

In [ ]:
train_df1 = pd.read_csv("../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv",skiprows=2000 ,header=None,nrows=20000)
train_df1.head()

We can mention no of rows to skip by using the option of **"skiprows"** as shown below with heading.Note that if we  want to import the headings from the csv file we need to skip first rows but use the first row for heading:

In [ ]:
train_df2 = pd.read_csv("../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv",skiprows=range(1,2000) ,nrows=20000)
train_df2.head()

# Technique 6: Use of Generators
Generators allow you to create a function that returns one item at a time rather than all the items at once. This means that if you have a large dataset, you don’t have to wait for the entire dataset to be accessible.

Generator functions allow you to declare a function that behaves like an iterator. It allows us to make an iterator in a fast, easy, and clean way. An iterator is an object that can be iterated (looped) upon. It is used to abstract a container of data to make it behave like an iterable object. Some examples of common iterable objects are strings, lists and dictionaries.

A generator looks a lot like a function, but uses the keyword **yield** instead of **return**. Let us take an example for better understanding. Suppose we try to generate numbers less than 10 using a generator by name "number_generator" as follows

In [ ]:
def number_generator():
    n = 0
    while n < 10:
        yield n
        n += 1
numbers = number_generator()
type(numbers)

That’s a generator function. So when we call it, it returns a generator object as shown above.

An important note here is how the state is encapsulated within the body of the generator function.To observe one by one value we can use the built-in **next()** function as follows

In [ ]:
next_no = number_generator()
next(next_no)


In [ ]:
next(next_no)

So you may wonder what will happen if we call **next()** past the end. For this we have a built in exception type called **StopIteration** that automatically raises once the generator stops yielding.

In summary when we call a **generator** function or use a generator expression, we return a special iterator called a generator. We can assign this generator to a variable in order to use it. When we call special methods on the generator, such as next(), the code within the function is executed up to yield.

Let us compare a normal approach vs using **generators** with respect to memory usage and time taken for the code to execute.

Suppose we iterate through a large list of numbers 500000 and store the square of all the numbers which are even in a seperate list.

### Normal Approach

In [ ]:
import memory_profiler
import time
def even_numbers(numbers):
    even = []
    for num in numbers:
        if num % 2 == 0: 
            even.append(num*num)
            
    return even
if __name__ == '__main__':
    m1 = memory_profiler.memory_usage()
    t1 = time.clock()
    cubes = even_numbers(range(500000))
    t2 = time.clock()
    m2 = memory_profiler.memory_usage()
    time_diff = t2 - t1
    mem_diff = m2[0] - m1[0]
    print(f"It took {time_diff} Secs and {mem_diff} Mb memory to execute Normal method")

### Generator Approach:
Now let us look at how efficient generator approach 

In [ ]:
import memory_profiler
import time
def even_numbers(numbers):
    for num in numbers:
        if num % 2 == 0:
            yield num * num 
    
if __name__ == '__main__':
    m1 = memory_profiler.memory_usage()
    t1 = time.clock()
    cubes = even_numbers(range(500000))
    t2 = time.clock()
    m2 = memory_profiler.memory_usage()
    time_diff = t2 - t1
    mem_diff = m2[0] - m1[0]
    print(f"It took {time_diff} Secs and {mem_diff} Mb memory to execute Generator method")

The advantage lies in the fact that generators don’t store all results in memory, rather they generate them on the fly, hence the memory is only used when we ask for the result. Also generators abstract away much of the boilerplate code needed when writing iterators, hence also helps in reducing the size of code.
# Technique 7: Eliminate unneccessary loops by using itertools
**itertools** saves you a lot of time on loops. It also gets rid of the complexity of the code.This module implements a number of iterator building blocks.

### Itertool functions

I will demostrating some of the itertool functions in this notebook .They provide streams of infinite length, so they should only be accessed by functions or loops that truncate the stream.

**itertools.accumulate(iterable[, func, *, initial=None])**

This iterator returns accumulated sums, or accumulated results of other binary functions (specified via the optional func argument).If func is supplied, it should be a function of two arguments. Elements of the input iterable may be any type that can be accepted as arguments to func.

Usually, the number of elements output matches the input iterable. However, if the keyword argument initial is provided, the accumulation leads off with the initial value so that the output has one more element than the input iterable.

Let us see an example to see how it works

In [ ]:
import operator
from itertools import product, chain
def accumulate(iterable, func=operator.add, *, initial=None):
    'Return running totals'
    # Example 1: accumulate([4,5,6,7,8,9]) --> 4 9 15 22 30 39
    # Example 2: accumulate([6,7,8,9], initial=1000) --> 1000 1006 1013 1021 
    # Example 3: accumulate([1,2,3,4,5], operator.mul) --> 1 2 6 24 120
    it = iter(iterable)
    total = initial
    if initial is None:
        try:
            total = next(it)
        except StopIteration:
            return
    yield total
    for element in it:
        total = func(total, element)
        yield total

Now let us see how we can multiply the values using the accumulate()

In [ ]:
data = [3, 4, 6, 2, 1, 9, 0, 7, 5, 8]
list(accumulate(data, operator.mul))     # running multiplication

Now let us see how we can maximum values using the accumulate()

In [ ]:
list(accumulate(data, max))              # running maximum

# Technique 8: Do not use + operator for strings

Start avoiding + operator for concatenation for strings as they are immutable,i.e every time we add an element to a string, Python creates a new string and a new address which means** new memory** needs to be allocated each time the string is altered.

Let us see with a simple example as shown below 

**Normal approach:**

In [ ]:
mymsg = "Micheal"
msg="My name is "+mymsg+". I live in US"
print(msg)

**New approach**

In [ ]:
mymsg = "Micheal"
msg="My name is %s . I live in US"% mymsg
print(msg)

* # Technique 9: Memory profiling

Juggling with large data sets involves having a clear sight of memory consumption and allocation processes going on in the background. There are tools to monitor the memory usage of your notebook.

Use **%memit** in familiar fashion to **%timeit**

Let us see this with an example .Suppose we want to caalculate PI value in a monte carlo simulation

In [ ]:
import numpy as np
from random import random
def pi_calculation(n=50000000) -> "area":
    """Estimate pi with monte carlo simulation.
    
    Arguments:
        n: Number of Simulations
    """
    return np.sum(np.random.random(n)**2 + np.random.random(n)**2 <= 1) / n * 4

Use this command to run the above function 
**%memit pi_calculation()**

 # Technique 10: Memory leaks
 
 We can avoid memory leaks in Jupyter Notebook by removing the temporary variables when we no longer need them.

In [ ]:
import pandas as pd

data = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

no_nans = data.dropna()
one_hot_encoded = pd.get_dummies(no_nans)
# some other temp variables
#processed_data = ...

del no_nans
del one_hot_encoded

That method is not recommended, because we can easily overlook a variable that should be removed or remove a used variable by mistake.

The better method of avoiding memory leaks is doing data processing inside a function. It creates a new scope for the intermediate variables and removes them automatically when the interpreter exits the function:

In [ ]:
import pandas as pd

def data_preprocessing(raw_data):
    no_nans = data.dropna()
    one_hot_encoded = pd.get_dummies(no_nans)
    # some other temp variables
    processed_data = 10
    return processed_data

data = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
processed_data = data_preprocessing(data)

# Technique 11: Line profiler

**%lprun** command yields the time spent on each line of code giving us a line by line report. Since not shipped by default, install the library with pip

In [ ]:
!pip install line_profiler

and load the extension manually in the notebook.

In [ ]:
%load_ext line_profiler

To demonstrate the capabilities of this feature let us define method which evaluates pi using random generated data points and then look for ways to optimize. Remember that the area covered by a circle with radius 1 inscribed in a square, equals exactly to a quarter of pi.

We get the value of pi by taking the ratio of area of circle to area of the square,

In [ ]:
from random import random

def estimate_pi(n=1e7) -> "area":
    """Estimate pi with monte carlo simulation.
    
    Arguments:
        n: number of simulations
    """
    in_circle = 0
    total = n
    
    while n != 0:
        prec_x = random()
        prec_y = random()
        if pow(prec_x, 2) + pow(prec_y, 2) <= 1:
            in_circle += 1 # inside the circle
        n -= 1
        
    return 4 * in_circle / total

 It is easy as pi(e) to locate hotspots in the code with only difference that functions need to be explicitly defined.

In [ ]:
%lprun -f estimate_pi estimate_pi()

Timer unit: 1e-06 s

Total time: 62.5456 s
File: <ipython-input-58-0f6e1f5ac99b>
Function: estimate_pi at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def estimate_pi(n=1e7) -> "area":
     4                                               """Estimate pi with monte carlo simulation.
     5                                               
     6                                               Arguments:
     7                                                   n: number of simulations
     8                                               """
     9         1          3.0      3.0      0.0      in_circle = 0
    10         1          1.0      1.0      0.0      total = n
    11                                               
    12  10000001    9739317.0      1.0     15.6      while n != 0:
    13  10000000   10079187.0      1.0     16.1          prec_x = random()
    14  10000000    9900919.0      1.0     15.8          prec_y = random()
    15  10000000   15220356.0      1.5     24.3          if pow(prec_x, 2) + pow(prec_y, 2) <= 1:
    16   7855684    7664704.0      1.0     12.3              in_circle += 1 # inside the circle
    17  10000000    9941136.0      1.0     15.9          n -= 1
    18                                                   
    19         1          3.0      3.0      0.0      return 4 * in_circle / total

Notice the extensive time (24.3%) spent on the if statement on line 15. Take in mind that we can even colorize the report above, giving us a more intuitive way to see **hot-spots**.
 
 All we have to do is insert the %%heat command at the top of the cell (to use load the extension%load_ext heat after installing with !pip install py-heat-magic) and it allows us to see the completely red while loop obliquing high cost of CPU-time, clearly showing room for optimization.
 
# Technique 12: Memory Profiler
In this technique, we will look at a simple **memory profiler** called memory_profiler. It is very similar to **line_profiler**, and it can be conveniently be used from IPython.The **memory_profiler** package checks the memory usage of the interpreter at every line. The increment column allows us to spot those places in the code where large amounts of memory are allocated. This is especially important when working with arrays. Unnecessary array creations and copies can considerably slow down a program. We will tackle this issue in the next few recipes.

Let us install it first

In [ ]:
!pip install memory_profiler

Now we load the memory_profiler IPython extension

In [ ]:
%load_ext memory_profiler

Now let us define a function that allocates large objects:

In [ ]:
%%writefile memscript.py
def myfunc():
    a = [1] * 5000000
    b = [2] * 7000000
    del b
    return a

**Now  let's run the code under the control of the memory profiler.**

In [ ]:
from memscript import myfunc
%mprun -T mprof0 -f myfunc myfunc()

**Finally let us see the outcome**

In [ ]:
print(open('mprof0', 'r').read())

# Technique 13: autoreload
The autoreload module is there to help with code structure. The module reloads the code before each execution. Once you get locked into a TDD loop and you start refactoring code from the notebook into additional files, this module will reload the code in the additional files.

In [ ]:
# Reload modules before executing user code
%load_ext autoreload
%autoreload 2

# setup backend for matplotlibs plots
%matplotlib inline


# Technique 14: Increase default memory limit size

Jupyter notebook has a default memory limit size. You can try to increase the memory limit by following the steps:

**1) Generate Config file using command:**

*jupyter notebook --generate-config*

**2) Open jupyter_notebook_config.py file situated inside 'jupyter' folder and edit the following property:**

*NotebookApp.max_buffer_size = your desired value*

Remember to remove the '#' before the property value.

**3) Save and run the jupyter notebook. It should now utilize the set memory value. Also, don't forget to run the notebook from inside the jupyter folder.**

Alternatively, you can simply run the Notebook using below command:

*jupyter notebook --NotebookApp.max_buffer_size=your_value*



### I hope you had a good insight on memory optimisation techniques mentioned above while handling large datasets(1GB or more) in this notebook and I am hopeful that these techniques comes handy in your day to day work. 

# Greatly appreciate to leave your comments /suggestions .
# If you like this notebook please do <font color='red'>UPVOTE .